**Reference: [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)**

Adapted from code contributed by **[donnemartin](https://github.com/donnemartin/data-science-ipython-notebooks/tree/master/deep-learning/keras-tutorial/deep_learning_models)**

In [8]:
import numpy as np
import warnings
from keras.models import Model
from keras.layers import Flatten,Dense,Input
from keras.layers import Convolution2D,MaxPooling2D
from keras.preprocessing import image
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils import get_file
from keras import backend as K

In [2]:
TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
def VGG19(include_top=True,weights='imagenet',input_tensor=None):
    '''# Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of 'None' (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of 'layers.Input()')
            to use as image input for the model.'''
    if weights not in {'imagenet',None}:
        raise ValueError("The 'weights' argument should be either 'None' (random initialization) or 'imagenet' (pre-training on ImageNet)")
    if include_top:
        input_shape = (224,224,3)
    else:
        input_shape = (None,None,3)
        
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor)
        else:
            img_input = input_tensor
            
    # Block 1
    x = Convolution2D(64,(3,3),activation='relu',padding='same',name='block1_conv1')(img_input)
    x = Convolution2D(64,(3,3),activation='relu',padding='same',name='block1_conv2')(x)
    x = MaxPooling2D((2,2),strides=(2,2),name='block1_pool')(x)
    
    # Block 2
    x = Convolution2D(128,(3,3),activation='relu',padding='same',name='block2_conv1')(x)
    x = Convolution2D(128,(3,3),activation='relu',padding='same',name='block2_conv2')(x)
    x = MaxPooling2D((2,2),strides=(2,2),name='block2_pool')(x)
    
    # Block 3
    x = Convolution2D(256,(3,3),activation='relu',padding='same',name='block3_conv1')(x)
    x = Convolution2D(256,(3,3),activation='relu',padding='same',name='block3_conv2')(x)
    x = Convolution2D(256,(3,3),activation='relu',padding='same',name='block3_conv3')(x)
    x = Convolution2D(256,(3,3),activation='relu',padding='same',name='block3_conv4')(x)
    x = MaxPooling2D((2,2),strides=(2,2),name='block3_pool')(x)
    
    # Block 4
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block4_conv1')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block4_conv2')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block4_conv3')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block4_conv4')(x)
    x = MaxPooling2D((2,2),strides=(2,2),name='block4_pool')(x)
    
    #Block 5
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block5_conv1')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block5_conv2')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block5_conv3')(x)
    x = Convolution2D(512,(3,3),activation='relu',padding='same',name='block5_conv4')(x)
    x = MaxPooling2D((2,2),strides=(2,2),name='block5_pool')(x)
    
    if include_top:
        x = Flatten(name='flatten')(x)
        x = Dense(4096,activation='relu',name='fc1')(x)
        x = Dense(4096,activation='relu',name='fc2')(x)
        x = Dense(1000,activation='softmax',name='predictions')(x)
        
    model = Model(img_input,x)
    
    if weights=='imagenet':
        print('K.image_dim_ordering:',K.image_dim_ordering())
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                   TF_WEIGHTS_PATH,
                                   cache_subdir='models')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                   TF_WEIGHTS_PATH_NO_TOP,
                                   cache_subdir='models')
            
        model.load_weights(weights_path)
    return model
    

In [5]:
def preprocess_input(x, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}

    if dim_ordering == 'th':
        x[:, 0, :, :] -= 103.939
        x[:, 1, :, :] -= 116.779
        x[:, 2, :, :] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, ::-1, :, :]
    else:
        x[:, :, :, 0] -= 103.939
        x[:, :, :, 1] -= 116.779
        x[:, :, :, 2] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]
    return x

In [6]:
import json
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'
def decode_predictions(preds):
    global CLASS_INDEX
    assert len(preds.shape) == 2 and preds.shape[1] == 1000
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    indices = np.argmax(preds, axis=-1)
    results = []
    for i in indices:
        results.append(CLASS_INDEX[str(i)])
    return results

In [7]:
model = VGG19(include_top=True,weights='imagenet')
img_path = 'strawberry.jpeg'
img = image.load_img(img_path,target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)
print('Input image shape:',x.shape)

preds = model.predict(x)
print('Predicted:',decode_predictions(preds))

K.image_dim_ordering: tf
574717952/574710816 [==============================] - 71s 0us/step
Input image shape: (1, 224, 224, 3)
Predicted: [['n07745940', 'strawberry']]
